In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from IPython.display import display
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Carregando o dataset
df = pd.read_csv('/content/Dataset_13_02.csv')

# Exibindo as primeiras linhas para confirmar que os dados foram carregados corretamente
print(df.head())

for coluna in df.select_dtypes(include="object").columns:  # Apenas colunas categóricas (strings)
    print(f"\nContagem para {coluna}:")
    print(df[coluna].value_counts())

# Contagem de dados faltantes em cada coluna
faltantes = df.isnull().sum()
print("Dados faltantes por coluna:")
print(faltantes)

# Mapeamento de benefícios
beneficio_map = {
    "A": "Programa Bolsa Família (PBF)",
    "B": "Benefício de Prestação Continuada (BPC/LOAS)",
    "C": "Bolsa ou benefício da Prefeitura Municipal",
    "D": "Bolsa ou benefício do Governo do Estado",
    "E": "Pensão",
    "F": "Aposentadoria",
    "G": "Outro benefício"
}

# Remover espaços extras dos valores na coluna 'Beneficios'
df["Beneficios"] = df["Beneficios"].astype(str).str.strip()

# Criar colunas binárias para cada benefício
for codigo in beneficio_map.keys():
    df[f"Beneficio_{codigo}"] = df["Beneficios"].apply(lambda x: 1 if codigo in x else 0)

# Criar uma coluna com a contagem total de benefícios por indivíduo
df["Total_Beneficios"] = df[[f"Beneficio_{codigo}" for codigo in beneficio_map.keys()]].sum(axis=1)

# Exibir as primeiras linhas do dataset transformado
print(df.head())

mapeamento = {
    "Região": {"Norte": 1, "Nordeste": 2, "Sudeste": 3, "Sul": 4, "Centro-Oeste": 5},
    "Sexo": {"Masculino": 1, "Feminino": 2},
    "Tipo de Domicílio": {"Casa": 1, "Apartamento": 2, "Outros": 3},
    "Possui Cozinha": {"Sim": 1, "Não": 0},
    "Ocupação": {"Próprio de algum morador - já pago": 1, "Próprio de algum morador - ainda pagando": 2,
                 "Alugado": 3, "Cedido por empregador": 4, "Cedido de outra forma": 5, "Outra condição": 6},
    "Situação do Registro": {"Urbano": 1, "Rural": 2},
    "Presença de Tosse": {"Sim": 1, "Não": 2, "Não sabe/ não quis responder": 9},
    "Tipo de Respiração": {"Sim": 1, "Não": 2, "Não sabe/ não quis responder": 9},
    "Alimentos Básicos": {"Não": 1, "Sim, raramente": 2, "Sim, às vezes": 3, "Sim, quase sempre": 4,
                          "Sim, sempre": 5, "Não se cozinha em casa": 6},
    "Nivel Escolaridade": {
        "Sem estudo": 0,
        "1° ano do ensino fundamental": 1,
        "1ª série/ 2°ano do ensino fundamental": 2,
        "2ª série/ 3°ano do ensino fundamental": 3,
        "3ª série/ 4°ano do ensino fundamental": 4,
        "4ª série/ 5°ano do ensino fundamental": 5,
        "5ª série/ 6°ano do ensino fundamental": 6,
        "6ª série/ 7°ano do ensino fundamental": 7,
        "7ª série/ 8°ano do ensino fundamental": 8,
        "8ª série/ 9°ano do ensino fundamental": 9,
        "1°ano do ensino médio": 10,
        "2°ano do ensino médio": 11,
        "3°ano do ensino médio": 12,
        "Ensino superior incompleto": 13,
        "Ensino superior completo": 14
    },
    "Faixa de Renda": {
        "Sem renda": 1, "Até R$ 1.000,00": 2, "De R$ 1.001,00 até R$ 2.000,00": 3,
        "De R$ 2.001,00 até R$ 3.000,00": 4, "De R$ 3.001,00 até R$ 5.000,00": 5,
        "De R$ 5.001,00 até R$ 10.000,00": 6, "R$ 10.001,00 ou mais": 7
    },
    "Cor Pessoa": {
        "Branca": 1,
        "Preta": 2,
        "Amarela (origem japonesa, chinesa, coreana etc.)": 3,
        "Parda (mulata, cabocla, cafuza, mameluca ou mestiça)": 4,
        "Indígena": 5,
        "Não sabe/não quis responder": 9
    },
    "Moradores que Alimentaram Acabamento (Sim)": {
        "Sim": 1, "Não": 2
    },
    "Moradores que Alimentaram Acabamento (Não)": {
        "Sim": 1, "Não": 2
    }
}

# Substituir valores da coluna "Idade em Meses", removendo a palavra "meses" e convertendo para número
coluna_idade_meses = "Idade em Meses"

# Verificar se a coluna existe no dataset antes de aplicar a transformação
if coluna_idade_meses in df.columns:
    df[coluna_idade_meses] = df[coluna_idade_meses].astype(str).str.replace("meses", "").str.strip()
    df[coluna_idade_meses] = pd.to_numeric(df[coluna_idade_meses], errors="coerce")  # Converter para número

# Aplicar os mapeamentos no dataset
for coluna, mapping in mapeamento.items():
    if coluna in df.columns:
        df[coluna] = df[coluna].map(mapping).fillna(9)

# Verificar dados faltantes após a transformação
faltantes = df.isnull().sum()
print("Dados faltantes por coluna após transformação:")
print(faltantes)

# Tratando dados faltantes em 'y' (Presença de Tosse)
imputer_y = SimpleImputer(strategy='most_frequent')
df['Presença de Tosse'] = imputer_y.fit_transform(df[['Presença de Tosse']]).ravel()

# Garantir que todas as colunas usadas no modelo sejam numéricas
X = df.drop(['Presença de Tosse', 'Data de Nascimento'], axis=1)  # Assumindo que você quer prever 'Presença de Tosse'
y = df['Presença de Tosse']

# Separar colunas numéricas e categóricas para imputação
numeric_columns = X.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = X.select_dtypes(include=['object']).columns

# Imputar valores faltantes para colunas numéricas com média
imputer_numeric = SimpleImputer(strategy='mean')
X[numeric_columns] = imputer_numeric.fit_transform(X[numeric_columns])

# Imputar valores faltantes para colunas categóricas com a moda (mais frequente)
imputer_categorical = SimpleImputer(strategy='most_frequent')
X[categorical_columns] = imputer_categorical.fit_transform(X[categorical_columns])

# Codificação de colunas categóricas que ainda não foram mapeadas
for col in categorical_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

# Usando SMOTE para balancear os dados, se necessário
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# Treinamento do modelo sem ajustes de hiperparâmetros
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Impressão das métricas de qualidade do modelo
y_pred = model.predict(X_test)
print("Acurácia do modelo:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Utilização de técnica para identificação dos melhores hiperparâmetros
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print("Melhores parâmetros:", grid_search.best_params_)

# Treinamento do modelo com os melhores hiperparâmetros
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Impressão das métricas de qualidade do modelo aperfeiçoado
y_pred_improved = best_model.predict(X_test)
print("Acurácia do modelo aperfeiçoado:", accuracy_score(y_test, y_pred_improved))
print(classification_report(y_test, y_pred_improved))

# Demonstração de uso do modelo com um dado novo
novo_dado = X_test.iloc[0].to_frame().T  # Pegando um exemplo do conjunto de teste para simular um novo dado
previsao = best_model.predict(novo_dado)
print(f'Predição para um dado exemplo: {"Sim" if previsao[0] == 1 else "Não"}')

# Nota: Para um dado verdadeiramente novo, você precisaria formatá-lo conforme o dataset original e aplicar as mesmas transformações de pré-processamento.



  Região       Sexo  Idade  Data de Nascimento Idade em Meses  \
0  Norte  Masculino      1            11092017       22 meses   
1  Norte   Feminino      4             6042015       51 meses   
2  Norte   Feminino      2            21122016       30 meses   
3  Norte  Masculino      4            13112014       56 meses   
4  Norte   Feminino      0             2092018       10 meses   

  Moradores que Alimentaram Acabamento (Sim)  \
0                                        Sim   
1                                        Sim   
2                                        Sim   
3                                        Sim   
4                                        Sim   

  Moradores que Alimentaram Acabamento (Não) Tipo de Domicílio Possui Cozinha  \
0                                        Não       Apartamento            Sim   
1                                        Não       Apartamento            Sim   
2                                        Sim              Casa            Sim

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Supondo que você já tenha as previsões do modelo aperfeiçoado
y_pred_improved = best_model.predict(X_test)

# Gerando a Matriz de Confusão
cm = confusion_matrix(y_test, y_pred_improved)

# Criando o display para plotar
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Não (0)", "Sim (1)"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Matriz de Confusão - Random Forest")
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Importâncias
importances = best_model.feature_importances_
feature_names = X.columns

# Ordenar da mais importante para a menos importante
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10,6))
plt.title("Importância das Features - Random Forest")
plt.bar(range(len(importances)), importances[indices], align="center")
plt.xticks(range(len(importances)), feature_names[indices], rotation=90)
plt.tight_layout()
plt.show()


# Carregamento e Inspeção de Dados:
- Importação de bibliotecas: São importadas bibliotecas essenciais do Python para manipulação de dados, visualização e aprendizado de máquina.
- Carregamento do dataset: O conjunto de dados é lido de um arquivo CSV.
- Inspecção dos dados: Exibe as primeiras linhas do dataset e verifica a frequência de variáveis categóricas.

# Pré-processamento de Dados:
- Tratamento de dados faltantes: Conta os valores ausentes para cada coluna.
- Mapeamento de benefícios: Cria colunas binárias para diferentes tipos de benefícios e soma-os em uma coluna de total de benefícios.
- Transformação de dados:
> - Define um dicionário de mapeamento para converter dados categóricos em valores numéricos para várias colunas.
> - Transforma 'Idade em Meses' removendo a string "meses" e convertendo para numérico.
> - Aplica os mapeamentos nas colunas relevantes.
# Limpeza de Dados:
- Imputação de valores faltantes:
> -Para a variável alvo 'Presença de Tosse', usa a estratégia de valor mais frequente.
> -Para outras características, separa colunas numéricas e categóricas e imputa valores faltantes com a média para as numéricas e a moda para as categóricas.
- Codificação: Utiliza LabelEncoder para quaisquer variáveis categóricas restantes não cobertas pelo mapeamento inicial.
# Preparação do Modelo:
- Divisão de Features e Target: Separa as características (X) e o alvo (y), excluindo 'Data de Nascimento', já que não é útil para o modelo.
# Balanceamento de Dados:
- SMOTE: Aplica SMOTE para equilibrar os dados, caso a distribuição das classes seja desigual.
# Treinamento e Avaliação do Modelo:
- Divisão dos dados: Divide os dados em conjuntos de treino e teste.
- Treinamento inicial: Treina um classificador RandomForest sem ajuste de hiperparâmetros.
- Métricas do modelo inicial: Calcula e imprime a acurácia e um relatório de classificação para o modelo inicial.
# Otimização do Modelo:
- Busca de hiperparâmetros: Usa GridSearchCV para encontrar os melhores hiperparâmetros do RandomForest.
- Treinamento com hiperparâmetros otimizados: Treina o modelo com os melhores hiperparâmetros encontrados.
- Métricas do modelo otimizado: Calcula e imprime a acurácia e o relatório de classificação para o modelo otimizado.
# Demonstração do Modelo:
- Previsão com dado novo: Demonstra como o modelo pode ser usado para prever com um exemplo de dado novo, notando que dados novos precisariam ser formatados e transformados conforme o dataset original.
Este código cobre um processo completo de preparação de dados, treinamento de modelo, otimização e avaliação para uma tarefa de classificação.